<a href="https://colab.research.google.com/github/kanpiromnida-cmd/madt7204-vibecoding/blob/main/%5B7204%5D_Google_Calendar_(Colab)_Edit_Update_Cancel_Sent_Events_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Calendar (Colab) — List & Add Events (08:00–18:00 only)

**Default timezone:** Asia/Bangkok  
**Create events only:** 08:00–18:00 (inclusive)  
**UI:** Tabs + Calendar dropdown (not only primary) + Attendees (email invites)

> Notes:
> - This notebook uses **OAuth client**.
> - Attendees emails are supported (multiple) and can send email invites automatically.

## 1) Install libraries

In [ ]:
# ติดตั้งไลบรารีที่จำเป็น (Colab บางทีมีอยู่แล้ว แต่ติดตั้งไว้ชัวร์)
!pip -q install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s eta 0:00:00


## 2) Authenticate user (Colab OAuth) + Connect to Google Calendar API

In [ ]:
from google.colab import auth

# ล็อกอินด้วยบัญชี Google ของคุณ (จะมี popup ให้กดอนุญาต)
auth.authenticate_user()
print("✅ Authenticated in Colab")

✅ Authenticated in Colab


In [ ]:
import google.auth
from googleapiclient.discovery import build

SCOPES = ["https://www.googleapis.com/auth/calendar"]  # read+write

creds, _ = google.auth.default(scopes=SCOPES)
service = build("calendar", "v3", credentials=creds, cache_discovery=False)

print("✅ Connected to Google Calendar API")

✅ Connected to Google Calendar API


In [ ]:
# !rm -rf /root/.config/gcloud
# print("✅ Cleared gcloud credentials cache")

## 3) Set Quota Project

Plase note that when using ADC (Application Default Credentials) on the Google Calendar API, you need to set a quota project.

Please replace `<YOUR_PROJECT_ID>` with your actual Google Cloud Project ID (e.g., `my-project-12345`) and run the cell below.

You can find your Project ID in the [Google Cloud Console](https://console.cloud.google.com/home/dashboard).

In [ ]:
!gcloud auth application-default login --scopes=https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/calendar


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  ํY
Please enter 'y' or 'n':  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=OfzZqFDda6RqNCZImlFtD3AZnC56NK&prompt=consent&token_usage=remote&access_type=offline&code_chall

In [ ]:
#@title 3.1) Set Quota Project
import subprocess

# project_id = input("Enter GCP Project ID: ").strip()
project_id = "tbprj-486006"  # <-- put your project ID here

if project_id:
    subprocess.run(
        ["gcloud", "auth", "application-default", "set-quota-project", project_id],
        check=True
    )
    print("✅ Quota project set")
else:
    print("❌ Project ID is empty. Please run again.")

✅ Quota project set




> ทำไมต้อง Enable API + ใส่ Project ID (Quota project)?

*   เพราะเวลาเรียก Google Calendar API มันต้อง “ผูกโควต้า/การคิดโควต้า” ไว้กับ Google Cloud Project สักอัน เพื่อป้องกัน abuse และนับ quota ถูกโปรเจกต์
*   ถ้าคุณใช้แนว ADC (Application Default Credentials) เช่น gcloud auth application-default ... หรือ google.auth.default()
→ มักจะต้องตั้ง quota project (ด้วย gcloud auth application-default set-quota-project ...) ไม่งั้นเจอ 403 แบบที่คุณเจอ (“quota project missing / accessNotConfigured”)





In [ ]:
# Setup (run once) > Connected to Google Calendar
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pytz

TZ_NAME = "Asia/Bangkok"
TZ = pytz.timezone(TZ_NAME)
SCOPES = ["https://www.googleapis.com/auth/calendar"]

def connect_service():
    auth.authenticate_user()
    creds, _ = google.auth.default(scopes=SCOPES)
    return build("calendar", "v3", credentials=creds, cache_discovery=False)

service = connect_service()
print("✅ Connected to Google Calendar")

/usr/local/lib/python3.12/dist-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.12/dist-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Connected to Google Calendar


# Google Calendar Tool

In [ ]:
#@title 📋 1) Today + List Events (Primary) — cleaner (EID stable)
from datetime import datetime, timedelta
from IPython.display import display, HTML
import zlib

LAST_LIST_START = None
LAST_LIST_END = None

def _card(msg, kind="info"):
    styles = {
        "ok":   "background:#f0fdf4;border:1px solid #bbf7d0;color:#065f46;",
        "warn": "background:#fff7ed;border:1px solid #fed7aa;color:#9a3412;",
        "err":  "background:#fef2f2;border:1px solid #fecaca;color:#b91c1c;",
        "info": "background:#eff6ff;border:1px solid #bfdbfe;color:#1e3a8a;",
    }
    display(HTML(f"""
    <div style="font-family:system-ui;border-radius:14px;padding:10px 12px;max-width:920px;{styles[kind]}">
      {msg}
    </div>
    """))

def _fmt_dt(x):
    if not x: return "-"
    if "T" in x:
        dt = datetime.fromisoformat(x.replace("Z","+00:00")).astimezone(TZ)
        return dt.strftime("%Y-%m-%d %H:%M")
    return f"{x} (all-day)"

def _range_inclusive(start_ymd: str, end_ymd: str):
    sd = TZ.localize(datetime.strptime(start_ymd, "%Y-%m-%d"))
    ed = TZ.localize(datetime.strptime(end_ymd, "%Y-%m-%d")) + timedelta(days=1)
    return sd, ed

def _fetch_events(sd_dt, ed_dt, max_results=50):
    items = []
    token = None
    while True:
        res = service.events().list(
            calendarId="primary",
            timeMin=sd_dt.isoformat(),
            timeMax=ed_dt.isoformat(),
            singleEvents=True,
            orderBy="startTime",
            maxResults=min(250, max_results),
            pageToken=token,
        ).execute()
        items.extend(res.get("items", []))
        token = res.get("nextPageToken")
        if not token or len(items) >= max_results:
            break
    return items[:max_results]

def _dedupe_display(items):
    out = []
    seen = set()
    for ev in items:
        summary = (ev.get("summary") or "").strip()
        st = ev.get("start", {}).get("dateTime") or ev.get("start", {}).get("date") or ""
        en = ev.get("end", {}).get("dateTime") or ev.get("end", {}).get("date") or ""
        key = (summary, st, en)
        if key in seen:
            continue
        seen.add(key)
        out.append(ev)
    return out

def _eid_from_event_id(event_id: str) -> int:
    # stable numeric id (0..4294967295)
    return zlib.crc32((event_id or "").encode("utf-8")) & 0xffffffff

def _render_table(items, title_text, subtitle, show_short_id=True):
    if not items:
        _card("No events found.", "warn")
        return

    rows = ""
    for ev in items:
        st = ev.get("start", {}).get("dateTime") or ev.get("start", {}).get("date")
        en = ev.get("end", {}).get("dateTime") or ev.get("end", {}).get("date")
        summary = ev.get("summary", "(no title)")
        st_fmt = _fmt_dt(st); en_fmt = _fmt_dt(en)
        time_col = "All-day" if "T" not in (st or "") else f"{st_fmt[11:16]}–{en_fmt[11:16]}"
        short_id = (ev.get("id","")[-6:] if show_short_id else "")
        eid = _eid_from_event_id(ev.get("id",""))

        id_col = f"<td style='padding:8px;border-bottom:1px solid #eef2f7;width:90px;color:#64748b;'>{short_id}</td>" if show_short_id else ""
        rows += f"""
        <tr>
          <td style="padding:8px;border-bottom:1px solid #eef2f7;width:130px;font-weight:800;color:#0f172a;">{eid}</td>
          {id_col}
          <td style="padding:8px;border-bottom:1px solid #eef2f7;width:120px;">{st_fmt[:10]}</td>
          <td style="padding:8px;border-bottom:1px solid #eef2f7;width:110px;">{time_col}</td>
          <td style="padding:8px;border-bottom:1px solid #eef2f7;">{summary}</td>
        </tr>
        """

    head_id = "<th style='padding:8px;'>Short ID</th>" if show_short_id else ""
    display(HTML(f"""
    <div style="font-family:system-ui;background:#fff;border:1px solid #e5e7eb;border-radius:16px;padding:14px 16px;max-width:920px;">
      <div style="font-weight:900;font-size:18px;color:#0f172a;">{title_text}</div>
      <div style="color:#64748b;margin-top:2px;">{subtitle}</div>
      <table style="width:100%;border-collapse:collapse;margin-top:10px;">
        <tr style="background:#f3f4f6;text-align:left;">
          <th style="padding:8px;">event_id (EID)</th>
          {head_id}
          <th style="padding:8px;">Date</th><th style="padding:8px;">Time</th><th style="padding:8px;">Title</th>
        </tr>
        {rows}
      </table>
      <div style="margin-top:10px;color:#64748b;font-size:12px;">
        👉 ใช้เลข <b>EID</b> ไปใส่ในเซลล์ “Manage by event_id” เพื่อ Update/Cancel (ไม่ต้องใช้ # แล้ว)
      </div>
    </div>
    """))

# --------- Form inputs ----------
run_now = True  #@param {type:"boolean"}
show_today = True #@param {type:"boolean"}
dedupe_display = True #@param {type:"boolean"}
show_short_id = True #@param {type:"boolean"}

list_start = "2026-02-03" #@param {type:"date"}
list_end   = "2026-02-28" #@param {type:"date"}
search_title = "" #@param {type:"string"}
max_show = 50 #@param {type:"integer"}

connect()

if not run_now:
    _card("✅ Ready. Set run_now = True แล้ว Run cell เพื่อดึงรายการ", "info")
else:
    if show_today:
        today = datetime.now(TZ).date().isoformat()
        sd, ed = _range_inclusive(today, today)
        today_items = _fetch_events(sd, ed, max_results=20)
        if dedupe_display:
            today_items = _dedupe_display(today_items)
        _render_table(today_items, "TODAY'S SCHEDULE", f"{today} · show up to 20", show_short_id=show_short_id)

    sd, ed = _range_inclusive(list_start, list_end)
    items = _fetch_events(sd, ed, max_results=max_show)

    q = search_title.strip().lower()
    if q:
        items = [ev for ev in items if q in (ev.get("summary") or "").lower()]

    if dedupe_display:
        items = _dedupe_display(items)

    LAST_LIST_START, LAST_LIST_END = list_start, list_end
    _render_table(items, "LIST EVENTS", f"{list_start} → {list_end} · max {max_show}" + (f" · filter: '{search_title}'" if q else ""),
                  show_short_id=show_short_id)

    if not dedupe_display:
        _card("ℹ️ ถ้าเห็นรายการหน้าตาเหมือนกันหลายอัน นั่นคือมี event ซ้ำจริงใน calendar (คนละ eventId)", "info")


✅ Already connected.


event_id (EID),Short ID,Date,Time,Title
27774496,6spgns,2026-02-12,09:00–10:00,Event Subject
2850373931,0vg0jo,2026-02-13,09:00–10:00,Consult/Feedback: MADT7204
529006334,cg40m0,2026-02-14,09:00–10:00,Event Subject
2072224150,i5m0o0,2026-02-20,09:00–10:00,Event Subject


In [ ]:
#@title ➕ 2) Quick Add & Notify (08:00–18:00) + agenda/description (confirm checkbox)
from datetime import datetime, timedelta
from IPython.display import display, HTML
import re

TIME_OPTS = [f"{h:02d}:{m:02d}" for h in range(8, 19) for m in (0,15,30,45)]
TIME_OPTS = [t for t in TIME_OPTS if t <= "18:00"]
EMAIL_RE = re.compile(r"[^@\s]+@[^@\s]+\.[^@\s]+")

def _pill(text):
    return f"<span style='display:inline-block;padding:6px 10px;border-radius:999px;background:#e0f2fe;color:#075985;font-weight:800;margin-right:8px;'>{text}</span>"

def _card_create_success(title, date, start_time, end_time, attendees_cnt, link):
    display(HTML(f"""
    <div style="font-family:system-ui;border-radius:18px;padding:16px 18px;max-width:920px;
                background:#f8fafc;border:1px solid #e5e7eb;">
      <div style="display:flex;align-items:center;gap:10px;">
        <div style="font-size:22px;">✅</div>
        <div style="font-size:22px;font-weight:900;color:#0f172a;">Event created successfully</div>
      </div>

      <div style="margin-top:8px;font-size:18px;font-weight:800;color:#0f172a;">
        {title} <span style="color:#64748b;font-weight:800;">— {date}</span>
      </div>

      <div style="margin-top:10px;">
        {_pill(f"{start_time}–{end_time}")}
        <span style="display:inline-block;padding:6px 10px;border-radius:999px;background:#e2e8f0;color:#334155;font-weight:800;">
          Attendees: {attendees_cnt}
        </span>
      </div>

      <div style="margin-top:14px;">
        <a href="{link}" target="_blank" style="text-decoration:none;font-weight:900;color:#2563eb;font-size:22px;">
          Open in Google Calendar →
        </a>
      </div>
    </div>
    """))

def _card(msg, kind="info"):
    styles = {
        "ok":   "background:#f0fdf4;border:1px solid #bbf7d0;color:#065f46;",
        "warn": "background:#fff7ed;border:1px solid #fed7aa;color:#9a3412;",
        "err":  "background:#fef2f2;border:1px solid #fecaca;color:#b91c1c;",
        "info": "background:#eff6ff;border:1px solid #bfdbfe;color:#1e3a8a;",
    }
    display(HTML(f"""
    <div style="font-family:system-ui;border-radius:14px;padding:10px 12px;max-width:920px;{styles[kind]}">
      {msg}
    </div>
    """))

def _to_bool(v):
    # กันเคส Colab ส่งค่าเป็น "False"/"True" (string textbox)
    if isinstance(v, bool):
        return v
    if v is None:
        return False
    s = str(v).strip().lower()
    return s in ("true", "1", "yes", "y", "on")

def _parse_attendees(raw):
    raw = (raw or "").strip()
    if not raw:
        return []
    parts = [p.strip() for p in raw.split(",") if p.strip()]
    bad = [p for p in parts if not EMAIL_RE.fullmatch(p)]
    if bad:
        raise ValueError("Invalid email(s): " + ", ".join(bad))
    seen=set(); out=[]
    for p in parts:
        k=p.lower()
        if k not in seen:
            seen.add(k); out.append({"email": p})
    return out

def _exists_duplicate(start_dt, end_dt, title_str):
    res = service.events().list(
        calendarId="primary",
        timeMin=(start_dt - timedelta(minutes=1)).isoformat(),
        timeMax=(end_dt + timedelta(minutes=1)).isoformat(),
        singleEvents=True,
        orderBy="startTime",
        maxResults=50,
    ).execute()
    for ev in res.get("items", []):
        if (ev.get("summary","").strip() == title_str.strip() and
            ev.get("start", {}).get("dateTime") == start_dt.isoformat() and
            ev.get("end", {}).get("dateTime") == end_dt.isoformat()):
            return True
    return False

# ---- Form fields ----
title = "Event Subject" #@param {type:"string"}
date  = "2026-02-20"    #@param {type:"date"}
start_time = "09:00"    #@param ["08:00","08:15","08:30","08:45","09:00","09:15","09:30","09:45","10:00","10:15","10:30","10:45","11:00","11:15","11:30","11:45","12:00","12:15","12:30","12:45","13:00","13:15","13:30","13:45","14:00","14:15","14:30","14:45","15:00","15:15","15:30","15:45","16:00","16:15","16:30","16:45","17:00","17:15","17:30","17:45","18:00"]
end_time   = "10:00"    #@param ["08:00","08:15","08:30","08:45","09:00","09:15","09:30","09:45","10:00","10:15","10:30","10:45","11:00","11:15","11:30","11:45","12:00","12:15","12:30","12:45","13:00","13:15","13:30","13:45","14:00","14:15","14:30","14:45","15:00","15:15","15:30","15:45","16:00","16:15","16:30","16:45","17:00","17:15","17:30","17:45","18:00"]
attendees = ""          #@param {type:"string"}
agenda_or_description = "" #@param {type:"string"}
send_invites = True     #@param {type:"boolean"}

# ⚠️ สำคัญ: ห้ามมี comment ต่อท้ายบรรทัดนี้ ไม่งั้น Colab อาจไม่เรนเดอร์เป็น checkbox
confirm_create = True  #@param {type:"boolean"}

connect()

_card("🛡️ Guardrail: create only 08:00–18:00 (15-min steps)", "warn")
_card("💡 agenda_or_description: พิมพ์ข้อความได้เลย (ไม่ต้องใส่ \" \") เช่น Agenda: ... หรือแปะลิงก์โน้ต", "info")

confirm_create = _to_bool(confirm_create)  # ✅ กันเคสดื้อเป็น textbox

if not confirm_create:
    _card("✅ Ready. ถ้าจะสร้าง event ให้ติ๊ก confirm_create แล้ว Run cell อีกครั้ง", "info")
else:
    t = (title or "").strip()
    desc = (agenda_or_description or "").strip()

    if not t:
        _card("Title is required.", "err")
    elif start_time not in TIME_OPTS or end_time not in TIME_OPTS:
        _card("Time must be within 08:00–18:00 (15-min steps).", "err")
    elif end_time <= start_time:
        _card("End time must be after start time.", "err")
    else:
        start_dt = TZ.localize(datetime.strptime(f"{date} {start_time}", "%Y-%m-%d %H:%M"))
        end_dt   = TZ.localize(datetime.strptime(f"{date} {end_time}", "%Y-%m-%d %H:%M"))

        if _exists_duplicate(start_dt, end_dt, t):
            _card("Duplicate found — event NOT created.", "warn")
        else:
            try:
                attendees_list = _parse_attendees(attendees)
            except Exception as e:
                _card(f"Attendees error: {e}", "err")
                attendees_list = None

            if attendees_list is not None:
                body = {
                    "summary": t,
                    "start": {"dateTime": start_dt.isoformat(), "timeZone": TZ_NAME},
                    "end":   {"dateTime": end_dt.isoformat(),   "timeZone": TZ_NAME},
                }
                if desc:
                    body["description"] = desc
                if attendees_list:
                    body["attendees"] = attendees_list

                created = service.events().insert(
                    calendarId="primary",
                    body=body,
                    sendUpdates="all" if (send_invites and attendees_list) else "none"
                ).execute()

                _card_create_success(
                    title=t,
                    date=date,
                    start_time=start_time,
                    end_time=end_time,
                    attendees_cnt=len(attendees_list),
                    link=created.get("htmlLink", "#")
                )


✅ Already connected.


In [ ]:
#@title 🧰 3) Manage by event_id (Update / Cancel) — safer (EID stable)
from datetime import datetime, timedelta
from IPython.display import display, HTML
import zlib

TIME_OPTS = [f"{h:02d}:{m:02d}" for h in range(8, 19) for m in (0,15,30,45)]
TIME_OPTS = [t for t in TIME_OPTS if t <= "18:00"]

def _card(msg, kind="info", link=None):
    styles = {
        "ok":   "background:#f0fdf4;border:1px solid #bbf7d0;color:#065f46;",
        "warn": "background:#fff7ed;border:1px solid #fed7aa;color:#9a3412;",
        "err":  "background:#fef2f2;border:1px solid #fecaca;color:#b91c1c;",
        "info": "background:#eff6ff;border:1px solid #bfdbfe;color:#1e3a8a;",
    }
    display(HTML(f"""
    <div style="font-family:system-ui;border-radius:14px;padding:10px 12px;max-width:920px;{styles[kind]}">
      {msg}
      {f"<div style='margin-top:6px;'><a href='{link}' target='_blank' style='color:#2563eb;font-weight:800;text-decoration:none;'>Open in Google Calendar →</a></div>" if link else ""}
    </div>
    """))

def _eid_from_event_id(event_id: str) -> int:
    return zlib.crc32((event_id or "").encode("utf-8")) & 0xffffffff

def _range_inclusive(start_ymd: str, end_ymd: str):
    sd = TZ.localize(datetime.strptime(start_ymd, "%Y-%m-%d"))
    ed = TZ.localize(datetime.strptime(end_ymd, "%Y-%m-%d")) + timedelta(days=1)
    return sd, ed

def _fetch_events(sd_dt, ed_dt, max_results=200):
    items = []
    token = None
    while True:
        res = service.events().list(
            calendarId="primary",
            timeMin=sd_dt.isoformat(),
            timeMax=ed_dt.isoformat(),
            singleEvents=True,
            orderBy="startTime",
            maxResults=min(250, max_results),
            pageToken=token,
        ).execute()
        items.extend(res.get("items", []))
        token = res.get("nextPageToken")
        if not token or len(items) >= max_results:
            break
    return items[:max_results]

def _fmt_dt(x):
    if not x: return "-"
    if "T" in x:
        dt = datetime.fromisoformat(x.replace("Z","+00:00")).astimezone(TZ)
        return dt.strftime("%Y-%m-%d %H:%M")
    return f"{x} (all-day)"

manage_action = "Cancel selected" #@param ["None","Update selected","Cancel selected"]
event_id = 2854437385 #@param {type:"integer"}  # ✅ EID จากตาราง
confirm_manage = True #@param {type:"boolean"}

# ช่วงค้นหา (สำคัญมาก): default ใช้ช่วงล่าสุดจากเซลล์ List ถ้ามี
manage_search_start = "2026-02-03" #@param {type:"date"}
manage_search_end   = "2026-02-28" #@param {type:"date"}

# update fields
new_date  = "2026-02-22" #@param {type:"date"}
new_start = "15:30" #@param ["08:00","08:15","08:30","08:45","09:00","09:15","09:30","09:45","10:00","10:15","10:30","10:45","11:00","11:15","11:30","11:45","12:00","12:15","12:30","12:45","13:00","13:15","13:30","13:45","14:00","14:15","14:30","14:45","15:00","15:15","15:30","15:45","16:00","16:15","16:30","16:45","17:00","17:15","17:30","17:45","18:00"]
new_end   = "16:00" #@param ["08:00","08:15","08:30","08:45","09:00","09:15","09:30","09:45","10:00","10:15","10:30","10:45","11:00","11:15","11:30","11:45","12:00","12:15","12:30","12:45","13:00","13:15","13:30","13:45","14:00","14:15","14:30","14:45","15:00","15:15","15:30","15:45","16:00","16:15","16:30","16:45","17:00","17:15","17:30","17:45","18:00"]

connect()

if manage_action == "None":
    _card("เลือก manage_action ก่อน (Update selected หรือ Cancel selected)", "info")
elif not confirm_manage:
    _card("✅ Ready. ถ้าจะทำจริงให้ตั้ง confirm_manage = True แล้ว Run cell อีกครั้ง", "info")
else:
    # ใช้ช่วงจากเซลล์ 1 ถ้ามี
    try:
        if "LAST_LIST_START" in globals() and LAST_LIST_START:
            manage_search_start = LAST_LIST_START
        if "LAST_LIST_END" in globals() and LAST_LIST_END:
            manage_search_end = LAST_LIST_END
    except Exception:
        pass

    sd, ed = _range_inclusive(manage_search_start, manage_search_end)
    items = _fetch_events(sd, ed, max_results=300)

    # หา event ที่ EID ตรงกัน
    target = None
    for ev in items:
        if _eid_from_event_id(ev.get("id","")) == int(event_id):
            target = ev
            break

    if not target:
        _card(
            f"หา EID={event_id} ไม่เจอในช่วง {manage_search_start} → {manage_search_end}<br>"
            f"👉 ลองขยาย manage_search_start/manage_search_end ให้ครอบคลุมวันที่ของ event",
            "err"
        )
    else:
        title = target.get("summary","(no title)")
        link = target.get("htmlLink")
        st = target.get("start", {}).get("dateTime") or target.get("start", {}).get("date")
        en = target.get("end", {}).get("dateTime") or target.get("end", {}).get("date")
        _card(
            f"Selected: <b>{title}</b><br>"
            f"{_fmt_dt(st)} → {_fmt_dt(en)}<br>"
            f"<span style='color:#64748b'>eventId(short)={target.get('id','')[-8:]}</span>",
            "info",
            link
        )

        if manage_action == "Cancel selected":
            service.events().delete(calendarId="primary", eventId=target["id"]).execute()
            _card(f"🗑️ Canceled: <b>{title}</b> (EID={event_id})", "ok", link)

        else:
            if new_end <= new_start:
                _card("New end time must be after new start time.", "err")
            elif new_start not in TIME_OPTS or new_end not in TIME_OPTS:
                _card("New time must be within 08:00–18:00 (15-min steps).", "err")
            else:
                new_start_dt = TZ.localize(datetime.strptime(f"{new_date} {new_start}", "%Y-%m-%d %H:%M"))
                new_end_dt   = TZ.localize(datetime.strptime(f"{new_date} {new_end}", "%Y-%m-%d %H:%M"))

                target["start"] = {"dateTime": new_start_dt.isoformat(), "timeZone": TZ_NAME}
                target["end"]   = {"dateTime": new_end_dt.isoformat(),   "timeZone": TZ_NAME}

                updated = service.events().update(calendarId="primary", eventId=target["id"], body=target).execute()
                _card(f"🕒 Rescheduled: <b>{title}</b> (EID={event_id})", "ok", updated.get("htmlLink"))


✅ Already connected.


# 📘 User Guide

<div style="font-family:system-ui;max-width:980px;margin:14px 0;">
  <div style="background:#0b1220;color:#fff;border-radius:18px;padding:16px 18px;border:1px solid #111827;">
    <div style="font-size:20px;font-weight:900;letter-spacing:.2px;">📘 User Guide — Google Calendar Mini Tool (Colab)</div>
    <div style="margin-top:6px;opacity:.8;font-weight:700;">Primary calendar only · Timezone: Asia/Bangkok · Guardrail: 08:00–18:00 (15-min steps)</div>
  </div>

  <div style="background:#fff;border:1px solid #e5e7eb;border-radius:18px;padding:14px 16px;margin-top:12px;">
    <div style="font-weight:900;color:#0f172a;font-size:16px;">✅ Before you start</div>
    <ul style="margin:8px 0 0 18px;color:#334155;line-height:1.6;">
      <li>Run the <b>connect()</b> cell first (Google Colab will ask for permission).</li>
      <li>This tool works on <b>primary calendar</b> only.</li>
      <li>Create/Update time is limited to <b>08:00–18:00</b> (15-minute steps).</li>
    </ul>
  </div>

  <div style="background:#f8fafc;border:1px solid #e5e7eb;border-radius:18px;padding:14px 16px;margin-top:12px;">
    <div style="font-weight:900;color:#0f172a;font-size:16px;">📋 Step 1 — Today + List Events</div>
    <ol style="margin:8px 0 0 18px;color:#334155;line-height:1.75;">
      <li>Set <b>show_today</b> (optional) to show today’s schedule.</li>
      <li>Select <b>list_start</b> and <b>list_end</b> to list events by date range.</li>
      <li>(Optional) Use <b>search_title</b> to filter by title keyword.</li>
      <li>Click ▶️ Run. You will see a table with <b>event_id (EID)</b> and <b>Short ID</b>.</li>
    </ol>
    <div style="margin-top:10px;padding:10px 12px;border-radius:14px;background:#eff6ff;border:1px solid #bfdbfe;color:#1e3a8a;">
      💡 <b>Tip:</b> ใช้ค่า <b>event_id (EID)</b> จากตาราง เพื่อไปใช้ใน Step 3 (Update/Cancel)
    </div>
  </div>

  <div style="background:#f8fafc;border:1px solid #e5e7eb;border-radius:18px;padding:14px 16px;margin-top:12px;">
    <div style="font-weight:900;color:#0f172a;font-size:16px;">➕ Step 2 — Quick Add & Notify</div>
    <ol style="margin:8px 0 0 18px;color:#334155;line-height:1.75;">
      <li>Fill <b>title</b>, <b>date</b>, <b>start_time</b>, <b>end_time</b>.</li>
      <li>(Optional) Add attendees: <b>attendees</b> = <code>a@x.com, b@y.com</code></li>
      <li>(Optional) Add agenda/notes/link in <b>agenda_or_description</b> (พิมพ์ได้เลย ไม่ต้องใส่เครื่องหมาย quote).</li>
      <li>If you want email invites, keep <b>send_invites</b> = True.</li>
      <li>Tick ✅ <b>confirm_create</b> then ▶️ Run again to create.</li>
    </ol>
    <div style="margin-top:10px;padding:10px 12px;border-radius:14px;background:#fff7ed;border:1px solid #fed7aa;color:#9a3412;">
      🛡️ <b>Safety:</b> ต้องติ๊ก <b>confirm_create</b> ก่อนทุกครั้ง เพื่อกันเผลอกดสร้าง event
    </div>
  </div>

  <div style="background:#f8fafc;border:1px solid #e5e7eb;border-radius:18px;padding:14px 16px;margin-top:12px;">
    <div style="font-weight:900;color:#0f172a;font-size:16px;">🧰 Step 3 — Manage by event_id (Update / Cancel)</div>
    <ol style="margin:8px 0 0 18px;color:#334155;line-height:1.75;">
      <li>Run Step 1 first to build the latest event list & mapping.</li>
      <li>Choose <b>manage_action</b> = Update selected / Cancel selected.</li>
      <li>Enter <b>event_id (EID)</b> from the table (not the # row number).</li>
      <li>For Update: set <b>new_date</b>, <b>new_start</b>, <b>new_end</b> (within 08:00–18:00).</li>
      <li>Tick ✅ <b>confirm_manage</b> then ▶️ Run again to execute.</li>
    </ol>
    <div style="margin-top:10px;padding:10px 12px;border-radius:14px;background:#fef2f2;border:1px solid #fecaca;color:#b91c1c;">
      ⚠️ <b>Warning:</b> Cancel will permanently delete the event from your calendar.
    </div>
  </div>

  <div style="background:#fff;border:1px solid #e5e7eb;border-radius:18px;padding:14px 16px;margin-top:12px;">
    <div style="font-weight:900;color:#0f172a;font-size:16px;">🧩 Troubleshooting</div>
    <ul style="margin:8px 0 0 18px;color:#334155;line-height:1.7;">
      <li><b>Nothing shows / permission error</b> → run <b>connect()</b> again or use the “Reconnect” cell (if you have one).</li>
      <li><b>Invite email not received</b> → email invites are sent only when <b>attendees</b> is not empty and <b>send_invites</b> is True.</li>
      <li><b>Duplicate found</b> → same title + same start/end already exists (tool prevents creating duplicates).</li>
      <li><b>confirm checkbox becomes textbox</b> → recreate the cell (delete → add new cell → paste again) to refresh Colab form rendering.</li>
    </ul>
  </div>

  <div style="margin-top:12px;padding:12px 14px;border-radius:18px;background:#f0fdf4;border:1px solid #bbf7d0;color:#065f46;">
    ✅ <b>Workflow summary:</b> Step 1 (List) → copy <b>event_id (EID)</b> → Step 3 (Update/Cancel) · Step 2 for quick add
  </div>
</div>
